In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-data/data/ani/cached_url_ani_sports.txt
/kaggle/input/news-data/data/ani/cached_url_ani_politics.txt
/kaggle/input/news-data/data/ani/cached_url_ani_entertainment.txt
/kaggle/input/news-data/data/ani/cached_url_ani_world.txt
/kaggle/input/news-data/data/ani/cached_url_ani_business.txt
/kaggle/input/news-data/data/hindustan_times/cached_url_ht_sports.txt
/kaggle/input/news-data/data/hindustan_times/cached_url_ht_india.txt
/kaggle/input/news-data/data/hindustan_times/cached_url_ht_world.txt
/kaggle/input/news-data/data/hindustan_times/cached_url_ht_business.txt
/kaggle/input/news-data/data/hindustan_times/cached_url_ht_entertainment.txt
/kaggle/input/news-data/data/ndtv/cached_url_ndtv_world.txt
/kaggle/input/news-data/data/ndtv/cached_url_ndtv_india.txt
/kaggle/input/news-data/data/ndtv/cached_url_ndtv_business.txt
/kaggle/input/news-data/data/times_now/cached_url_tn_world.txt
/kaggle/input/news-data/data/times_now/cached_url_tn_business.txt
/kaggle/input/news-data/da

In [3]:
!pip install nltk
!pip install newspaper3k
!pip install PIL
!pip install matplotlib

     |████████████████████████████████| 211 kB 404 kB/s eta 0:00:01
     |████████████████████████████████| 7.4 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 3.0 MB/s  eta 0:00:01
     |████████████████████████████████| 80 kB 4.4 MB/s  eta 0:00:01
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398406 sha256=e0395e079c28661f779f44ec7934a83a1a77ece429fcb48d263d64f32b9392fd
  Stored in directory: /root/.cache/pip/wheels/4c/91/46/3c208287b726df325a5979574324878b679116e4baae1af3c3
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=bd4a3a1a8cb3ded92c8fad99b11e4af68a751c18f842d23332df107e836e102f
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3355 sha256=70239268148e6f7023ae170931e90da64fbb23a43e8f6f2027983abeda31fb93
  Stored in directory: /root/.cache/pi

In [4]:
# !pip install bert-extractive-summarizer
# !pip install transformers==2.2.0
# !pip install spacy==2.0.12

In [6]:
import nltk
from newspaper import Config, Article, Source
#nltk.download('punkt')
import re
newsid = 1
# from summarizer import Summarizer,TransformerSummarizer

In [7]:
import csv
with open('./newsdata.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["News_ID", "Website", "Category", "Headline", "URL","Top_Image","Date", "author"])
    
# with open('./summarydata.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(["News_ID", "Newspaper3k", "BERT", "GPT-2", "XLNET"])
    

In [8]:
directory = '/kaggle/input/news-data/data'
import os
folders = []
names = []
files = []
for x in os.walk(directory):
    folders.append(x[0])
    names.append(x[1])
    files.append(x[2])
folders.pop(0)
names = names[0]
files.pop(0)

[]

In [9]:
def get_url(location):
    urls = []
    with open(location, 'r') as filehandle:
            for line in filehandle:
                # remove linebreak which is the last character of the string
                currentPlace = line[:-1]

                # add item to the list
                urls.append(currentPlace)
    return urls

In [10]:
def summary(body):
    bert_model = Summarizer()
    bert_summary = ''.join(bert_model(body, num_sentences=3))
    #print(bert_summary)
    
    GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
    gpt2_summary = ''.join(GPT2_model(body, num_sentences=2))
    #print(gpt2_summary)

    xlnet_model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")
    xlnet_summary = ''.join(model(body,  num_sentences=2))
    #print(xlnet_summary)
    
    yield bert_summary, gpt2_summary, xlnet_summary

In [11]:
def data_loading(name, category, urls):
    global newsid
    config = Config()
    config.MAX_SUMMARY_SENT = 3
    for url in urls:
        #Get the article
        article = Article(url, config = config)
        article.download() #Downloads the link’s HTML content
        try:
            article.parse() #Parse the article
            article.nlp()#  Keyword extraction wrapper
            author = article.authors
            date = article.publish_date
            image_url = article.top_image
            headline = article.title
            body = article.text
            newspaper3k_summary = article.summary
            #insert these features inside a spreadsheet
            with open('./newsdata.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([newsid, name, category, headline, url, image_url, date, author])

            #call summarizer function which will create a new spreadsheet with 4 summaries from newspaper3k, BERT, GPT-2, XLNET
            #summarizer(url,newsid)
#             bert_summary, gpt2_summary, xlnet_summary = summary(body)
#             with open('./summarydata.csv', 'a', newline='') as file:
#                 writer = csv.writer(file)
#                 writer.writerow([newsid, newspaper3k_summary, bert_summary, gpt2_summary, xlnet_summary])
            

            newsid += 1
        except :
            pass

In [12]:
def data_gen(x, folder):
    global file
    for file in files[x]:
        path = folder+'/'+file
        category = re.findall('sports|business|politics|entertainment|india|world', file)
        urls = get_url(path)
        yield category, urls

In [13]:
def get_url_gen(location):
    with open(location, 'r') as filehandle:
            for line in filehandle:
                # remove linebreak which is the last character of the string
                currentPlace = line[:-1]
                yield currentPlace
    

In [14]:
for x in range(len(names)):
    name = names[x]
    name = name.replace('_', ' ')
    folder = folders[x]
    for file in files[x]:
        path = folder+'/'+file
        category = re.findall('sports|business|politics|entertainment|india|world', file)
        url = get_url_gen(path)
        data_loading(name, category, url)
        